In [10]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

base_url = 'https://open.dosm.gov.my'

In [16]:
r = requests.get('https://open.dosm.gov.my/data-catalogue')

In [17]:
soup = BeautifulSoup(r.content)
data = soup.find_all('a', {'class': 'text-primary underline hover:no-underline'})
urls = [urljoin(base_url, d.get('href')) for d in data]
len(urls), urls[0]

(264,
 'https://open.dosm.gov.my/data-catalogue/dosm-public-economy_fuelprice_3')

In [90]:
url

'https://open.dosm.gov.my/data-catalogue/dosm-public-healthcare_covid-cases_5'

In [18]:
import sqlite3

connection = sqlite3.connect('opendosm.db')

In [127]:
from tqdm import tqdm
directory = 'csv'

os.makedirs(directory, exist_ok = True)

maps = {}
for url in tqdm(urls):
    r_ = requests.get(url)
    name = url.split('/')[-1]
    name = name.replace('-', ' ').replace('_', ' ').split()
    name = ''.join([n.title() for n in name])
    
    soup = BeautifulSoup(r_.content)
    csv_url = [a.get('href') for a in soup.find_all('a') if a.get('href').endswith('csv')][0]
    script_tag = soup.find('script', {'id': '__NEXT_DATA__'})
    data = json.loads(str(script_tag).split('">')[1].split('</script>')[0])
    explanation = data['props']['pageProps']['explanation']
    metadata = data['props']['pageProps']['metadata']
    df = pd.read_csv(csv_url)
    df.to_sql(name=name, con=connection, if_exists = 'replace')
    
    filename = f'{name}.csv'
    df.to_csv(os.path.join(directory, filename), index = False)
    maps[url] = {'explanation': explanation, 'metadata': metadata, 'filename': filename}

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 264/264 [03:56<00:00,  1.12it/s]


In [128]:
with open('opendosm.json', 'w') as fopen:
    json.dump(maps, fopen)